# Basic Logging

Logging is a means of tracking events that happen when some software runs. The software’s developer adds logging calls to their code to indicate that certain events have occurred. An event is described by a descriptive message which can optionally contain variable data (i.e. data that is potentially different for each occurrence of the event). Events also have an importance which the developer ascribes to the event; the importance can also be called the level or severity.

## When to print vs. logging?

Logging is typically written to a different location and is accessable by looking in that different location. Printing is printed to the screen while the program is running. But did you know there are differet locations to send a print statement? What, it all goes to the terminal what do you mean?

A terminal has at least two standard locations to send information (i.e. text): Standard Out, Standard Error. 

When we use the print() function the default location is standard out.

In [1]:
print("Hello World")

Hello World


The print() function can take a multitude of objects and corretly convert them into text before sending to standard out. In the previous example "Hello World" is a single string. In the following example there are two strings. There is no space in the text between "Hello" and "World" but print() adds one as the default.

In [2]:
print("Hello", "World")

Hello World


But we can also define the separatur to use.

In [3]:
print("Hello", "World", sep='|')

Hello|World


There is also an assumed line return added to the end. We can define the end value and override the default.

In [4]:
print("Hello world", end='')
print("Good by")

Hello worldGood by


In [5]:
import sys

In fact print() is just a wrapper around the system call to write to standard out. We can go directly to the system standard out. In this example "Hello world" is printed and the function call returned the length of the string, 11. This is then printed as well.

In [6]:
sys.stdout.write("Hello world")

Hello world

11

So we could also write directly to standard error. Since both streams print to the terminal (or notebook in this case) they will appear to be almost the same. Since there was something printed to standard error the notebook highlights that text in pink signifying something mostlikely went wrong. Typically when we want to convey some information about a problem it is sent to standard error. When it is just information it should go to standard out.

In [7]:
sys.stderr.write("An error occured")

An error occured

16

We can change where the standard out or error is sent. In this case we will redirect it to a variable for standard error. Notice how the text is not printed to the screen and there is no pink highlighted text. We have captured the text into the variable _standard_error_

In [8]:
from io import StringIO
orig_stderr = sys.stderr
standard_error = StringIO()
sys.stderr = standard_error
sys.stderr.write("An error occured")

16

If we print that variable it is not actually a variable. We need to get the value before it becomes a Python string variable.

In [9]:
standard_error

In [10]:
result_string_stderr = standard_error.getvalue()
result_string_stderr

'An error occured'

We can return standar error back to the orginal location and now everything sent to standard error will be printed to the notebook.

In [11]:
sys.stderr = orig_stderr

In [12]:
sys.stderr.write("An error occured")

An error occured

16

We could also redirect all standard out to a file and access it later.

In [13]:
orig_stdout = sys.stdout
sys.stdout = open('stdout_file.out', 'w')
print('Hello world')

In [14]:
sys.stdout = orig_stdout

Here we will search the current directory for a file with the name we created. It finds one file.

In [15]:
from pathlib import Path
files = list(Path.cwd().glob('stdout_file.out'))
print(files)

[PosixPath('/Users/kehoe/Git_area/AtmosphericPythonCourse/cool_stuff/stdout_file.out')]


We can print the contents. Notice it reads in the full string incdluding the line return which is "\n".

In [16]:
files[0].read_text()

'Hello world\n'

Let's clean up the file we just created by removing it.

In [17]:
Path(files[0]).unlink()

Knowing this, you would be tempted to redirect all standard out and standard error to a file and call it good, right? Well not exactly. We can get better control over what is printed to a file or terminal by using logging. Logging provides a set of convenience functions for simple logging usage. These are debug(), info(), warning(), error() and critical(). Using the different functions controls which will be printed depending on the settings.

Here is an example where we can use logging to differentiate which messages are printed and which are not. The default is to print warning() and more sever, but not to print info() or debug(). So only one of the messages is printed. Notice that the color of the text box is pink. This means the output is going to standard error. This will go to the terimal just like standard out but it looks like a normal error/warning.

In [18]:
import logging
logging.warning('Watch out!')  # will print a message to the console
logging.error("It's comming right for you!")  # will print a message to the console
logging.critical('You are going to get hit!')  # will print a message to the console
logging.info('I told you so')  # will not print anything
logging.debug('You never listen to me')  # will not print anything

ERROR:root:It's comming right for you!
CRITICAL:root:You are going to get hit!


Typicaly you will begin by setting the level to receive the information sent by each of the functons. The level is bases on the 5 functions and setting the level to DEBUG includes all levels below. To set the level after running the cell above we need to reload the module. But if we did not execute the cell above this would not be necessay.

In [19]:
import importlib
importlib.reload(logging)  # Need to reload the module
logging.basicConfig(level=logging.DEBUG)

In [20]:
logging.warning('Watch out!')  # will print a message to the console
logging.error("It's comming right for you!")  # will print a message to the console
logging.critical('You are going to get hit!')  # will print a message to the console
logging.info('I told you so')  # will print a message to the console
logging.debug('You never listen to me')  # will print a message to the console

ERROR:root:It's comming right for you!
CRITICAL:root:You are going to get hit!
INFO:root:I told you so
DEBUG:root:You never listen to me


But often you will send the output to a file instead of standard error. This is done during the configuration process.

In [21]:
importlib.reload(logging)  # Need to reload the module
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)
logging.warning('Watch out!')  # will print a message to the file
logging.error("It's comming right for you!")  # will print a message to the file
logging.critical('You are going to get hit!')  # will print a message to the file
logging.info('I told you so')  # will print a message to the file
logging.debug('You never listen to me')  # will print a message to the file

Now we will read the file we just created and print. It is all text and the returns are not followed so contains "\n" but is not utilzed.

In [22]:
Path('example.log').read_text()

"WARNING:root:Watch out!\nERROR:root:It's comming right for you!\nCRITICAL:root:You are going to get hit!\nINFO:root:I told you so\nDEBUG:root:You never listen to me\n"

Let's reset the location the logging information is sent and clean up the file we just created. All events at or above DEBUG level will now get logged.

In [23]:
Path('example.log').unlink(missing_ok=True)  # Delete log file from current directory.

importlib.reload(logging)  # Need to reload the module to allow set/reset level.
logging.basicConfig(level=logging.DEBUG)

We can send variable information to the logging with some standard Python syntax. 

In [24]:
a = 'Look'
b = 'leap!'
logging.warning('%s before you %s', a, b)

The format of the output has a default. We can update that to our needs. Here we set the output format to not include logger name.

In [25]:
importlib.reload(logging)  # Need to reload the module to allow set/reset level.
logging.basicConfig(format='%(levelname)s:%(message)s')  # Reset the format

In [26]:
logging.warning('%s before you %s', a, b)

It is very common to start each line of a log file with date/time to aid in debugging. Since this log file may be used for a long time this helps us determine which log messages are relevent to our issue. The format of the _datefmt_ argument is the same as supported by time.strftime().

In [27]:
importlib.reload(logging)  # Need to reload the module to allow set/reset level.
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S.%s %Z')
logging.warning('is when this event was logged.')

2024-02-14 14:07:28.1707944848 MST is when this event was logged.


You can use any of the Python string formatting methods when creating the message to log/print. In this example we use f-string formatting to insert the variable name into the message.

In [28]:
importlib.reload(logging)  # Need to reload the module to allow set/reset level.
logging.basicConfig(format='%(asctime)s %(message)s')
name = 'John'
logging.warning(f'{name} has entered the building.')

2024-02-14 14:07:28,279 John has entered the building.


For larger more complex programs with different modules it may be important to indicate where the logging was called. This can be done through the stack trace. This example will show which function had the error (in this case the ipykernal_*) and the line number. Since we did not define the asctime format it will use the default. It is important to set exc_info=True. If not the information will be much less useful.

In [29]:
try:
  c = 5 / 0
except Exception as e:
  logging.error("Exception occurred", exc_info=True)

2024-02-14 14:07:28,283 Exception occurred
Traceback (most recent call last):
  File "/var/folders/08/hzthl3yd3mqd30ltf7nr9yfr0000gq/T/ipykernel_4492/2415565439.py", line 2, in <module>
    c = 5 / 0
ZeroDivisionError: division by zero


In [30]:
try:
  c = 5 / 0
except Exception as e:
  logging.error("Exception occurred")

2024-02-14 14:07:28,286 Exception occurred


In general if you are logging from an exception handler there is a method set up to call logging.error() with exc_info=True.

In [31]:
try:
  c = 5 / 0
except Exception as e:
  logging.exception("Exception occurred")

2024-02-14 14:07:28,290 Exception occurred
Traceback (most recent call last):
  File "/var/folders/08/hzthl3yd3mqd30ltf7nr9yfr0000gq/T/ipykernel_4492/2319844081.py", line 2, in <module>
    c = 5 / 0
ZeroDivisionError: division by zero


So far we have been using the defualt logger named _root_. You can and should define your own logger by creating an object of the Logger class. Notice how the output is different than the standard. This is becuase the name of the custom logger is not part of the default output. But we can create the same output format by setting the format.

In [32]:
importlib.reload(logging)  # Need to reload the module to allow set/reset level.
logger = logging.getLogger('example_logger')
logger.warning('This is a warning')
logger = logging.getLogger(__name__)
logger

This is a warning


<Logger __main__ (WARNING)>

Custom handlers require using handlers to set the format and optioins of our logger. Handlers come into the picture when you want to configure your own loggers and send the logs to multiple places when they are generated. Handlers send the log messages to configured destinations like the standard output stream or a file or over HTTP or to your email via SMTP.

A logger that you create can have more than one handler, which means you can set it up to be printed to the terminal and saved to a log file and also send it over email.

Like loggers, you can also set the severity level in handlers. This is useful if you want to set multiple handlers for the same logger but want different severity levels for each of them. For example, you may want logs with level WARNING and above to be logged to the console, but everything with level ERROR and above should also be saved to a file. Here’s a program that does that.

I this example we set up the logger to print both WARNING and ERROR to the terminal but also send ERROR to a log file. Since the log file works by appending the new message, each time we run the cell below we add a new line to the log file.

In [33]:
importlib.reload(logging)  # Need to reload the module to allow set/reset level.

# Create a custom logger using the name of the current program/file.
logger = logging.getLogger(__name__)

# Create handlers
c_handler = logging.StreamHandler()  # This is a StreamHandler and prints to the consol.
f_handler = logging.FileHandler('file.log')  # This is a FileHandler and prints to a file.
c_handler.setLevel(logging.WARNING)  # Sets logging at WARNGING and above to go to consol.
f_handler.setLevel(logging.ERROR)  # Sets logging at ERROR and above to go to file.

# Create formatters and add it to handlers. This is what the message will look like when printed.
c_format = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
c_handler.setFormatter(c_format)
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(c_handler)
logger.addHandler(f_handler)

logger.warning('This is a warning')
logger.error('This is an error')

__main__ - WARNING - This is a warning
__main__ - ERROR - This is an error


If we look for a file in the current directory with the name _file.log_ it will contain the ERROR message but not the WARNING message because we set only ERROR and above to go to the file. But we set WARNING and above to also be printed to consol. Since we have different formats based on where the message is printed we get a different format for the _file.log_.

In [34]:
Path('file.log').read_text()  # Read the file and print

'2024-02-14 14:07:28,301 - __main__ - ERROR - This is an error\n'

In [35]:
Path('file.log').unlink(missing_ok=True)  # Delete log file from current directory.

We can also specify the logger parameters with a configuration file and read that file in. There is a file called _logger_configuration.conf_ in the current directory with configuration information. We can initialize that by asking the logging function to read the file and apply the settings.

In [36]:
import logging.config

logging.config.fileConfig(fname='logger_configuration.conf', disable_existing_loggers=False)

# Get the logger specified in the file
logger = logging.getLogger(__name__)

logger.debug('This is a debug message')

2024-02-14 14:07:28,315 - __main__ - DEBUG - This is a debug message


This is a lot of information. Don't think you understand it and know what to do. Chances are you don't but at least you know some of what is possble and if you are struggling you know there are more options. Implement logging and use it.